# Gendered Reception of Politicians in Online Political Discourse

### Computational Social Sciences

**Authors:** BELLAIS Salome, GONZALEZ DARDIK Micaela Natali, MARCULESCU Tudor, RODRIGUEZ Miguel, VIELLARD Mathilde.
**Course:** Computational Social Sciences (2025–2026)

---

## Abstract

Online social media platforms are key arenas for political debate, shaping public perception of political figures. While prior research highlights that female politicians often face different forms of evaluation and criticism than their male counterparts—frequently involving gendered language and personal attacks—data availability can limit broad generalizations.

In this project, we focus on the online reception of selected French political figures, primarily Marine Le Pen and Emmanuel Macron, with additional data on a male politician from a similar ideological background. Using computational methods, we analyze textual content and interaction patterns on Twitter to examine differences in sentiment, toxicity, thematic focus, and network dynamics.

By combining natural language processing techniques with graph-based analysis, this study investigates how gender and political alignment intersect in shaping online discourse, while explicitly acknowledging the methodological and ethical limitations inherent in computational social science approaches.